Importing Libraries and Setting Directory

In [ ]:
import time
import datetime
import pandas as pd
import os
import numpy as np
import yfinance as yf

# Set directory to where the Excel files are stored
os.chdir('C:\\Users\\odeya.h\\SNC Dropbox\\Odeya Hazani Cohen\\פרוייקט')


Loading Data from Excel Files

In [ ]:
nasdaq_startups = pd.read_excel('tickers.xlsx', 'nasdaq startups (2)')
nasdaq_tickers = nasdaq_startups['ticker'].dropna().unique().tolist()
indexes = pd.read_excel('tickers.xlsx', 'indexes')
index_tickers = indexes['ticker'].dropna().unique().tolist()


Defining Date Range and Creating Date DataFrame

In [ ]:
start_date = '2019-01-01'
end_date = '2024-06-1'

# Create a DataFrame with all dates in the range
days = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D', name='Date'))
days['Date'] = pd.to_datetime(days['Date']).dt.date


Setting Up Excel Writer and DataFrames for Calculations

In [ ]:
# Excel writer to save outputs
xlwriter = pd.ExcelWriter('calculate data.xlsx', engine='openpyxl')

# DataFrame to store calculated market caps
index_values = pd.DataFrame(days['Date'])
original_values = pd.DataFrame(days['Date'])
market_caps = pd.DataFrame(days['Date'])


Initializing Lists for Failed Tickers

In [ ]:
failed_indexes = []
failed_tickers = []


Creating Dictionary for Market Cap DataFrames

In [ ]:
primary_categories = nasdaq_startups['primary'].unique()
primary_market_caps = {category: pd.DataFrame(days['Date']) for category in primary_categories}


Processing Index Data

In [ ]:
for index_ticker in index_tickers:
    try:
        df_index = yf.download(index_ticker, start=start_date, end=end_date)
        df_index.reset_index(inplace=True)
        df_index['Date'] = pd.to_datetime(df_index['Date']).dt.date
        index_values = index_values.merge(df_index[['Date', 'Adj Close']].rename(columns={'Adj Close': index_ticker}), on='Date', how='left')
    except Exception as e:
        failed_indexes.append(index_ticker)


Processing NASDAQ Ticker Data

In [ ]:
# Process NASDAQ ticker data
for nasdaq_ticker in nasdaq_tickers:
    try:
        df_ticker = yf.download(nasdaq_ticker, start=start_date, end=end_date)
        df_ticker.reset_index(inplace=True)
        df_ticker['Date'] = pd.to_datetime(df_ticker['Date']).dt.date
        shares = nasdaq_startups.loc[nasdaq_startups['ticker'] == nasdaq_ticker, 'shares_outstanding'].iloc[0]

        # Store original and calculate market cap
        original_values = original_values.merge(df_ticker[['Date', 'Adj Close']].rename(columns={'Adj Close': nasdaq_ticker}), on='Date', how='left')

        df_ticker['Market Cap'] = df_ticker['Adj Close'] * shares
        market_caps = market_caps.merge(df_ticker[['Date', 'Market Cap']].rename(columns={'Market Cap': nasdaq_ticker}), on='Date', how='left')

        # Get the primary category for the current ticker
        primary_category = nasdaq_startups.loc[nasdaq_startups['ticker'] == nasdaq_ticker, 'primary'].iloc[0]
        primary_market_caps[primary_category] = primary_market_caps[primary_category].merge(
            df_ticker[['Date', 'Market Cap']].rename(columns={'Market Cap': nasdaq_ticker}), on='Date', how='left')
        
    except Exception as e:
        failed_tickers.append(nasdaq_ticker)


Defining Function to Process Each Primary Category DataFrame

In [ ]:
# Function to process each primary category DataFrame
def process_category_df(df, category_name):
    # Calculate the sum of values in each row, excluding the 'Date' column specifically
    df['number of companies'] = df.drop('Date', axis=1).count(axis=1)
    df['Total MC- Finder'] = df.select_dtypes(include=[np.number]).sum(axis=1)
    
    # Remove rows where the Total Market Cap column is zero 
    df_filtered = df[df['Total MC- Finder'] != 0]

    # Initialize an empty list to store the summary values
    summary_values = []

    # Ensure df_filtered is a copy of the original DataFrame slice
    df_filtered = df_filtered.copy()

    # Get only the columns that are not 'Date' and not calculated columns
    non_date_columns = df_filtered.columns.difference(['Date', 'Total MC- Finder', 'number of companies', 'new tickers sum', 'left tickers sum', 'MC without new companies', 'denominator', 'index'])

    # Iterate over each row in the DataFrame
    for i in range(len(df_filtered)):
        row = df_filtered[non_date_columns].iloc[i]
        
        if i == 0:
            relevant_cells = row.isnull()
        else:
            prev_row = df_filtered[non_date_columns].iloc[i - 1]
            relevant_cells = row.notnull() & prev_row.isnull()
        
        summary_value = row[relevant_cells].sum()
        summary_values.append(summary_value)

    # Add the summary values as a new column using .loc
    df_filtered.loc[:, 'new tickers sum'] = summary_values

    # Initialize an empty list to store the summary values
    summary_values = []

    # Iterate over each row in the DataFrame
    for i in range(len(df_filtered)):
        row = df_filtered[non_date_columns].iloc[i]
        
        if i == 0:
            relevant_cells = row.isnull()
        else:
            prev_row = df_filtered[non_date_columns].iloc[i - 1]
            relevant_cells = row.isnull() & prev_row.notnull()
        
        summary_value = row[relevant_cells].sum()
        summary_values.append(summary_value)

    # Add the summary values as a new column using .loc
    df_filtered.loc[:, 'left tickers sum'] = summary_values

    # Set the value in 'left tickers sum', 'new tickers sum' at the first index to zero
    df_filtered.loc[0, 'left tickers sum'] = 0
    df_filtered.loc[0, 'new tickers sum'] = 0

    # Adding 'MC without new companies' column
    df_filtered['MC without new companies'] = df_filtered['Total MC- Finder'] - df_filtered['new tickers sum']

    # Adding 'denominator' column
    denominator = []
    for i in range(len(df_filtered)):
        row = df_filtered.iloc[i]
        if i == 0:
            denominator.append(1000000000)
        else:
            last_d = denominator[i-1]
            calc = (last_d * df_filtered['Total MC- Finder'].iloc[i]) / df_filtered['MC without new companies'].iloc[i]
            denominator.append(calc)

    df_filtered.loc[:, 'denominator'] = denominator     

    # Adding 'index' column
    df_filtered['index'] = df_filtered['Total MC- Finder'] / df_filtered['denominator']

    # Handle NaNs properly without filling them with zeros
    df_filtered['highest value ticker'] = df_filtered[non_date_columns].apply(lambda x: x.idxmax() if not x.isnull().all() else np.nan, axis=1)
    df_filtered['highest value level'] = df_filtered[non_date_columns].max(axis=1, skipna=True)
    df_filtered['average'] = df_filtered[non_date_columns].mean(axis=1, skipna=True)
    df_filtered['median'] = df_filtered[non_date_columns].median(axis=1, skipna=True)

    # Write the processed DataFrame to a separate sheet
    sheet_name = f'{category_name[:28]}_MC'  # Shorten to 28 characters and add _MC
    df_filtered.to_excel(xlwriter, sheet_name=sheet_name, index=False)


Processing and Saving Each Primary Category DataFrame

In [ ]:
for category, df in primary_market_caps.items():
    process_category_df(df, category)


Processing and Saving Overall Market Caps

In [ ]:
process_category_df(market_caps, 'Overall Market Caps')

Writing DataFrames to Separate Sheets in Excel Workbook

In [ ]:
index_values.to_excel(xlwriter, sheet_name='NASDAQ Index', index=False)
pd.DataFrame(failed_indexes, columns=['Failed Indexes']).to_excel(xlwriter, sheet_name='Failed Indexes')
pd.DataFrame(failed_tickers, columns=['Failed Tickers']).to_excel(xlwriter, sheet_name='Failed Tickers')


Closing the Excel Writer

In [ ]:
xlwriter.close()